<a href="https://colab.research.google.com/github/ChrisW19/llm_engineering/blob/main/Week3Day5Mod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai gradio httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 847.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# imports

import os
import torch
import requests
import gradio as gr
import re
from openai import OpenAI

from IPython.display import Markdown, display, update_display
from google.colab import drive, userdata
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig


In [4]:
# Model Constants

AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [5]:
# Authentication

hf_token = userdata.get("HF_TOKEN")
openai_api_key = userdata.get('OPENAI_API_KEY')
if not hf_token or not openai_api_key:
    raise ValueError("Missing HF_TOKEN or OPENAI_API_KEY. Set them as environment variables.")

login(hf_token, add_to_git_credential=True)
openai = OpenAI(api_key=openai_api_key)


In [6]:
# Tokenizer Setup

tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [7]:
# Model Quantization for Performance Optimization

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [8]:
# Load Model Efficiently

device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [9]:
# Function to trancribe audio file

def transcribe_audio(audio_file):
  with open(audio_file, "rb") as file:
    try:
      transcription = openai.audio.transcriptions.create(
          model="whisper-1", file=file, response_format="text"
      )
      return transcription if transcription else "Transcription failed."
    except Exception as e:
      return f"Error during transcription: {e}"

In [10]:
#Generate Meeting Minutes

def generate_meeting_minutes(transcription):
    system_message = (
        "You are an AI assistant that generates structured meeting minutes from transcripts."
        " Provide a summary with attendees, location, date; discussion points; key takeaways; "
        "and action items (with owners) in markdown format."
    )

    user_prompt = f"Transcript:\n{transcription}\n\nGenerate structured minutes in markdown."

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
      ]

    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda", non_blocking=True)

    max_tokens = min(2000, len(transcription.split()) * 2)  # Adjust max tokens based on input size

    with torch.no_grad():
      outputs = model.generate(
          inputs,
          max_new_tokens=max_tokens,
          pad_token_id=tokenizer.eos_token_id
      )

    minutes = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Post-processing: Remvoed redundent trancription text
    if transcription[:50] in minutes:
      minutes = minutes.replace(transcription, "").strip()

    return minutes or "Minutes generation failed"

In [11]:
def process_audio(file_path):
    transcription = transcribe_audio(file_path)
    if "Error" in transcription:
      return transcription, "Meeting minutes generation failed due to transcription error."
    minutes = generate_meeting_minutes(transcription)
    return transcription, minutes

In [12]:
# Gradio interface
with gr.Blocks() as iface:
    gr.Markdown("# AI Meeting Minutes Generator")
    gr.Markdown("Upload an audio file, and the AI will transcribe it and generate structured meeting minutes.")

    audio_input = gr.Audio(type="filepath", label="Upload Audio File")
    submit_button = gr.Button("Generate Transcript and Meeting Minutes")

    with gr.Row():
      with gr.Column():
          gr.Markdown("### Transcription")
          transcription_box = gr.Textbox(label="Transcription", lines=10)

      with gr.Column():
          gr.Markdown("### Meeting Minutes")
          minutes_box = gr.Markdown(label="Meeting Minutes")

    submit_button.click(
        process_audio,
        inputs=audio_input,
        outputs=[transcription_box, minutes_box]
    )

iface.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3f14d8d74d8fcb3d64.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3f14d8d74d8fcb3d64.gradio.live
